# Rethinking Statistics course in NumPyro - Week 7

Lecture 13: Monsters & Mixtures (Poisson GLMs, survival, zero-inflation)

- [Video](https://www.youtube.com/watch?v=p7g-CgGCS34)
- [Slides](https://speakerdeck.com/rmcelreath/l13-statistical-rethinking-winter-2019)

Lecture 14: Ordered Categories, Left & Right

- [Video](https://www.youtube.com/watch?v=zA3Jxv8LOrA)
- [Slides](https://speakerdeck.com/rmcelreath/l14-statistical-rethinking-winter-2019)

[Proposed problems](https://github.com/gbosquechacon/statrethinking_winter2019/blob/master/homework/week07.pdf) and [solutions in R](https://github.com/gbosquechacon/statrethinking_winter2019/blob/master/homework/week07_solutions.pdf) for the exercises of the week.

In [1]:
import pandas as pd
import numpy as np

from jax import random
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from numpyro.distributions.transforms import OrderedTransform
from numpyro.infer import NUTS, MCMC, Predictive

In [2]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Sat Mar 27 2021

Python implementation: CPython
Python version       : 3.9.1
IPython version      : 7.21.0

numpyro: 0.6.0
pandas : 1.2.3
jax    : 0.2.10
numpy  : 1.20.1

Watermark: 2.2.0



In [3]:
rng_key = random.PRNGKey(0)

## Exercise 1

> In the Trolley data, `Trolley`, we saw how education level (modeled as an ordered category) is associated with responses. Is this association causal? One plausible confound is that education is also associated with age, through a causal process: People are older when they finish school than when they begin it.

> Reconsider the `Trolley` data in this light. Draw a DAG that represents hypothetical causal relationships among response, education, and age. Which statical model or models do you need to evaluate the causal influence of education on responses? Fit these models to the trolley data. What do you conclude about the causal relationships among these three variables?

This is my DAG:

<img src="./fig/w7_img1.png" width="30%">

Age could influence both response `R` and education `E`. It could influence `R`, because people at different ages could have different attitudes. Age could influence education, because the longer you have lived, the more education you could have completed (up to a point). It's like the age causing marriage example from earlier in the course.

To evaluate the causal influence of `E` on `R`, we need to block the back-door from `E` through `A` to `R`. So we need a model that conditions on both `E` and `A`. Then the estimate for `E` should be the causal influence of `E`.

In [4]:
d = pd.read_csv('./dat/Trolley.csv', header=0, sep=';')
elvl = d['edu'].unique()
idx = [7 , 0 , 6 , 4 , 2 , 1, 3, 5]
cat = pd.Categorical(d.edu, categories=list(elvl[idx]), ordered=True)
d['edu_cat'] = pd.factorize(cat, sort=True)[0].astype('int')
d['age_std'] = (d.age - d.age.mean())/d.age.std()
d.tail(3)

,case,response,order,id,age,male,edu,action,intention,contact,story,action2,edu_cat,age_std
9927,ilshi,7,7,98;299,66,1,Graduate Degree,0,1,0,shi,0,7,2.003041
9928,ilswi,2,18,98;299,66,1,Graduate Degree,0,1,0,swi,0,7,2.003041
9929,nfrub,2,17,98;299,66,1,Graduate Degree,1,0,0,rub,1,7,2.003041


In [5]:
# https://fehiepsi.github.io/rethinking-numpyro/12-monsters-and-mixtures.html

def model(action, contact, intention, age_std, edu_cat, response=None):
    # priors
    cutpoints = numpyro.sample('cutpoints', 
                               dist.TransformedDistribution( 
                                   dist.Normal(0, 1.5).expand([6]), 
                                   OrderedTransform()
                                   ),
                               )
    norm = lambda label: numpyro.sample(label, dist.Normal(0,0.5))
    bA = norm('bA')
    bC = norm('bC')
    bI = norm('bI')
    bIA = norm('bIA')
    bIC = norm('bIC')
    bAge = norm('bAge')
    bEdu = norm('bEdu')
    edu_coefs = numpyro.sample('edu_coefs', dist.Dirichlet(jnp.repeat(2,7)))
    # likelihood
    delta_j = jnp.pad(edu_coefs, (1, 0))
    delta_E = jnp.sum(jnp.where(jnp.arange(8) <= edu_cat[..., None], delta_j, 0), -1)
    BI = bI + bIA*action + bIC*contact
    phi = bA*action + BI*intention + bC*contact + bAge*age_std + bEdu*delta_E
    response_hat = numpyro.sample('response_hat', dist.OrderedLogistic(phi, cutpoints), obs=response-1)

In [6]:
kernel = NUTS(model)
mcmc = MCMC(kernel, num_warmup=500, num_samples=500, num_chains=4, chain_method='sequential')
dat = {k:v.to_numpy() for k,v in d[['action', 'contact', 'intention', 'age_std', 'edu_cat', 'response']].items()}
mcmc.run(rng_key, **dat)
mcmc.print_summary()

sample: 100%|██████████| 1000/1000 [00:53<00:00, 18.66it/s, 63 steps of size 9.42e-02. acc. prob=0.95]



                  mean       std    median      5.0%     95.0%     n_eff     r_hat
          bA     -0.48      0.05     -0.48     -0.57     -0.39   1383.00      1.00
        bAge     -0.10      0.02     -0.10     -0.13     -0.06    582.99      1.00
          bC     -0.35      0.07     -0.35     -0.47     -0.24   1340.60      1.00
        bEdu      0.21      0.14      0.23      0.04      0.41    235.69      1.01
          bI     -0.30      0.06     -0.30     -0.40     -0.20   1188.45      1.00
         bIA     -0.43      0.08     -0.43     -0.55     -0.30   1289.67      1.00
         bIC     -1.24      0.10     -1.24     -1.39     -1.07   1182.44      1.00
cutpoints[0]     -2.51      0.11     -2.49     -2.65     -2.32    279.62      1.01
cutpoints[1]     -1.81      0.11     -1.80     -1.94     -1.63    264.82      1.01
cutpoints[2]     -1.21      0.11     -1.20     -1.34     -1.03    267.12      1.01
cutpoints[3]     -0.17      0.11     -0.16     -0.32     -0.01    267.94      1.02
cut

You may recall from the chapter that education has a negative effect in the model without age. Now that we include age, education has a positive influence (with some overlap with zero). So age has indeed soaked up some of the previous influence assigned to education. The back-door may be real.

I'd summarize this model, assuming this DAG is true, as saying that age causes people to give slightly lower responses. This could be a cohort effect, and not a causal influence of age. Either way, it is small. Education seems to cause higher responses (more approval). This suggests that education trains people to see some or all of the features A, I, C as more permissible. A model that interacted education with each might shed more light on things. Remember: A DAG doesn't say whether you need an interaction effect or not. That is a separate problem.

## Exercise 2

> Consider one more variable in the Trolley data: Gender. Suppose that gender might influence education as well as response directly. Draw the DAG now that includes response, education, age, and gender.

> Using only the DAG, is it possible that the inferences from Problem 1 are confounded by gender? If so, define any additional models you need to infer the causal influence of education on response. What do you conclude?

This is my DAG:

<img src="./fig/w7_img2.png" width="30%">

Here's the model we need, which includes education, age, and gender (female dummy variable):

In [7]:
d = pd.read_csv('./dat/Trolley.csv', header=0, sep=';')
elvl = d['edu'].unique()
idx = [7 , 0 , 6 , 4 , 2 , 1, 3, 5]
cat = pd.Categorical(d.edu, categories=list(elvl[idx]), ordered=True)
d['edu_cat'] = pd.factorize(cat, sort=True)[0].astype('int')
d['age_std'] = (d.age - d.age.mean())/d.age.std()
d.tail(3)

,case,response,order,id,age,male,edu,action,intention,contact,story,action2,edu_cat,age_std
9927,ilshi,7,7,98;299,66,1,Graduate Degree,0,1,0,shi,0,7,2.003041
9928,ilswi,2,18,98;299,66,1,Graduate Degree,0,1,0,swi,0,7,2.003041
9929,nfrub,2,17,98;299,66,1,Graduate Degree,1,0,0,rub,1,7,2.003041


In [8]:
def model(action, contact, intention, age_std, edu_cat, male, response=None):
    # priors
    cutpoints = numpyro.sample('cutpoints', 
                               dist.TransformedDistribution( 
                                   dist.Normal(0, 1.5).expand([6]), 
                                   OrderedTransform()
                                   ),
                               )
    norm = lambda label: numpyro.sample(label, dist.Normal(0,0.5))
    bA = norm('bA')
    bC = norm('bC')
    bI = norm('bI')
    bIA = norm('bIA')
    bIC = norm('bIC')
    bAge = norm('bAge')
    bEdu = norm('bEdu')
    edu_coefs = numpyro.sample('edu_coefs', dist.Dirichlet(jnp.repeat(2,7)))
    bMale = norm('bMale')
    # likelihood
    delta_j = jnp.pad(edu_coefs, (1, 0))
    delta_E = jnp.sum(jnp.where(jnp.arange(8) <= edu_cat[..., None], delta_j, 0), -1)
    BI = bI + bIA*action + bIC*contact
    phi = bA*action + BI*intention + bC*contact + bAge*age_std + bEdu*delta_E + bMale*male
    response_hat = numpyro.sample('response_hat', dist.OrderedLogistic(phi, cutpoints), obs=response-1)

In [9]:
kernel = NUTS(model)
mcmc = MCMC(kernel, num_warmup=500, num_samples=500, num_chains=4, chain_method='sequential')
dat = {k:v.to_numpy() for k,v in d[['action', 'contact', 'intention', 'age_std', 'edu_cat', 'male', 'response']].items()}
mcmc.run(rng_key, **dat)
mcmc.print_summary()

sample: 100%|██████████| 1000/1000 [01:18<00:00, 12.67it/s, 63 steps of size 8.40e-02. acc. prob=0.95]



                  mean       std    median      5.0%     95.0%     n_eff     r_hat
          bA     -0.48      0.05     -0.48     -0.57     -0.39   1275.96      1.00
        bAge     -0.07      0.02     -0.07     -0.11     -0.03    934.27      1.00
          bC     -0.35      0.07     -0.35     -0.47     -0.25   1279.55      1.00
        bEdu     -0.00      0.18      0.01     -0.28      0.27    469.78      1.01
          bI     -0.29      0.06     -0.29     -0.39     -0.20   1195.19      1.00
         bIA     -0.44      0.08     -0.44     -0.58     -0.31   1220.66      1.00
         bIC     -1.26      0.10     -1.26     -1.42     -1.10   1207.35      1.00
       bMale      0.57      0.04      0.57      0.51      0.63   1696.12      1.00
cutpoints[0]     -2.37      0.14     -2.36     -2.58     -2.15    523.24      1.01
cutpoints[1]     -1.67      0.14     -1.66     -1.91     -1.47    508.45      1.01
cutpoints[2]     -1.07      0.14     -1.06     -1.29     -0.86    509.96      1.01
cut

Age is still negative (and weak), while education is right near zero and straddles both sides. Gender seems to have accounted for all of the previous influenced assigned to education. It looks like female respondents gave lower average responses, indicating less approval.

It would be worth figuring out how gender is associated with education in this sample. It could be true for example that some education levels under-sampled men or women, and this leads to another kind of confound. Consider for example if older men are less likely to respond, so the sample becomes increasingly female with age. Then education level will also be increasingly female with age. Since the sample is
not a representative sample of the population, there are probably some biases of this sort.